# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [201]:
# Write your code below.
%load_ext dotenv
%dotenv ../../01_materials/labs/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [202]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [203]:
import os
from glob import glob

# Write your code below.

import os
from glob import glob

price_data_dir = os.getenv("PRICE_DATA")

price_data_all_parquet_glob = glob(price_data_dir + '**/*.parquet', recursive=True)

price_data_files_only_glob = [file for file in price_data_all_parquet_glob if os.path.isfile(file)]

price_data_files_only_glob


['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [204]:
# Write your code below.
import dask.dataframe as dd

dd_price_data = dd.read_parquet(price_data_files_only_glob).set_index("ticker")

dd_feat = dd_price_data.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), 
                       Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: (x['Adj Close']/x['Adj_Close_lag']) - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
)

dd_feat



c:\Users\nalca\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
C:\Users\nalca\AppData\Local\Temp\ipykernel_54248\3534232673.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_price_data.groupby('ticker', group_keys=False).apply(


Dask DataFrame Structure:
                            Date     Open     High      Low    Close Adj Close Volume  sector subsector   year Close_lag Adj_Close_lag  returns hi_lo_range
npartitions=5749                                                                                                                                           
                  datetime64[ns]  float64  float64  float64  float64   float64  int64  object    object  int32   float64       float64  float64     float64
                             ...      ...      ...      ...      ...       ...    ...     ...       ...    ...       ...           ...      ...         ...
...                          ...      ...      ...      ...      ...       ...    ...     ...       ...    ...       ...           ...      ...         ...
                             ...      ...      ...      ...      ...       ...    ...     ...       ...    ...       ...           ...      ...         ...
                             ...      ...      ...      ...      ...       ...    ...     ...       ...    ...       ...           ...      ...         ...
Dask Name: assign, 18 graph layers

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [205]:
# Write your code below.
import pandas as pd

pdf_feat = dd_feat.compute()

pdf_feat.reset_index(inplace=True)

pdf_feat['rolling_average'] = pdf_feat.groupby('ticker', group_keys=False)['returns'].rolling(10).mean().reset_index(level=0, drop=True)

pdf_feat


ticker       Date       Open       High        Low      Close  \
0          MMM 2001-01-02  49.383362  50.794315  49.122074  49.827549   
1          MMM 2001-01-03  49.331104  50.114967  48.651756  49.252716   
2          MMM 2001-01-04  49.278847  49.958195  48.312080  49.749165   
3          MMM 2001-01-05  48.913044  48.913044  46.221783  47.894024   
4          MMM 2001-01-08  47.240803  48.233696  45.803722  48.181438   
...        ...        ...        ...        ...        ...        ...   
1425342     ED 2011-12-23  61.799999  62.189999  61.689999  61.980000   
1425343     ED 2011-12-27  61.980000  62.540001  61.840000  62.500000   
1425344     ED 2011-12-28  62.500000  62.549999  62.139999  62.189999   
1425345     ED 2011-12-29  62.189999  62.669998  62.189999  62.590000   
1425346     ED 2011-12-30  62.669998  62.740002  62.009998  62.029999   

         Adj Close    Volume       sector                 subsector  year  \
0        25.659340   5562835  Industrials  Industrial Conglomerates  2001   
1        25.363338   5686262  Industrials  Industrial Conglomerates  2001   
2        25.618998   4856238  Industrials  Industrial Conglomerates  2001   
3        24.663656  10006932  Industrials  Industrial Conglomerates  2001   
4        24.811665   6561974  Industrials  Industrial Conglomerates  2001   
...            ...       ...          ...                       ...   ...   
1425342  38.161835   1110600    Utilities           Multi-Utilities  2011   
1425343  38.481998    852100    Utilities           Multi-Utilities  2011   
1425344  38.291145    879400    Utilities           Multi-Utilities  2011   
1425345  38.537422    935900    Utilities           Multi-Utilities  2011   
1425346  38.192619   1189400    Utilities           Multi-Utilities  2011   

         Close_lag  Adj_Close_lag   returns  hi_lo_range  rolling_average  
0              NaN            NaN       NaN     1.672241              NaN  
1        49.827549      25.659340 -0.011536     1.463211              NaN  
2        49.252716      25.363338  0.010080     1.646114              NaN  
3        49.749165      25.618998 -0.037290     2.691261              NaN  
4        47.894024      24.663656  0.006001     2.429974              NaN  
...            ...            ...       ...          ...              ...  
1425342  61.619999      37.940178  0.005842     0.500000         0.004575  
1425343  61.980000      38.161835  0.008390     0.700001         0.006123  
1425344  62.500000      38.481998 -0.004960     0.410000         0.005083  
1425345  62.189999      38.291145  0.006432     0.480000         0.006301  
1425346  62.590000      38.537422 -0.008947     0.730003         0.003756  

[1425347 rows x 16 columns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    - No, it is also possible to perform the calculation in Dask
+ Would it have been better to do it in Dask? Why?
    - Depending on the need, Dask would be better to ensure computational efficiency, but if we want to just add the rolling average on the data without re-running compute, pandas is a good option. This is also dependent on the amount of data, if we know that the data would be too large for in-memory computations, then Dask would almost always be better.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [x] Created a branch with the correct naming convention.
- [x] Ensured that the repository is public.
- [x] Reviewed the PR description guidelines and adhered to them.
- [x] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.